In [2]:
%matplotlib inline

# imports
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import os

import neurodsp as ndsp
from scv_funcs import lfpca
from scv_funcs import utils

CKEYS = plt.rcParams['axes.prop_cycle'].by_key()['color']
font = {'family' : 'arial',
        'weight' : 'regular',
        'size'   : 15}
import matplotlib
matplotlib.rc('font', **font)

In [54]:
from bokeh.plotting import figure, output_file, save

output_file("kjm_bp.html")

In [19]:
# electrode location
elec_def = {
    '1': 'dorsal M1',
    '3': 'dorsal S1',
    '4': 'ventral S1+M1',
    '6': 'frontal (non-R)',
    '7': 'parietal (non-R)',
    '8': 'temporal',
    '9': 'occipital'
}
fs = 1000.

In [3]:
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from scipy.stats import expon
output_notebook()

Loading BokehJS ...

In [5]:
cd results/kjm_digits/bp/

C:\Users\Lauren\Voytek\spectralCV\results\kjm_digits\bp


In [21]:
trial_data = np.load('trial_info.npz')

In [22]:
trial_info = trial_data['trial_info']
elec_regions = trial_data['elec_regions']

In [6]:
ls

 Volume in drive C is OS
 Volume Serial Number is B431-6EAA

 Directory of C:\Users\Lauren\Voytek\spectralCV\results\kjm_digits\bp

09/04/2018  03:35 PM    <DIR>          .
09/04/2018  03:35 PM    <DIR>          ..
09/04/2018  03:35 PM         7,952,706 move.npz
09/04/2018  03:35 PM         7,952,706 pre.npz
09/04/2018  03:35 PM             5,480 trial_info.npz
09/04/2018  03:35 PM        44,311,106 whole.npz
               4 File(s)     60,221,998 bytes
               2 Dir(s)  294,567,792,640 bytes free


In [14]:
lfpca_all = []
lfpca_all.append(lfpca.lfpca_load_spec('pre.npz'))

In [16]:
chan_sel = []
for plot_chan in range(lf.numchan):
    try:
        ele_name = elec_def[elec_regions[plot_chan].astype(str)]
    except:
        ele_name = 'others'
    chan_sel.append('%i: '%plot_chan+ele_name)
    
def update_spct_hist(f=10, channel=0, numbins=20):
    plot_chan = int(channel.split(':')[0])
    plot_freq = np.where(lf.f_axis==f)[0][0]
    y, x = np.histogram(lf.spg[plot_chan,plot_freq,:], bins=numbins, density=True)
    # update histogram with data from frequency f
    hist_plot.data_source.data['left'] = x[:-1]
    hist_plot.data_source.data['right'] = x[1:]
    hist_plot.data_source.data['top'] = y    
    # update fitted
    rv = expon(scale=sp.stats.expon.fit(lf.spg[plot_chan,plot_freq,:],floc=0)[1])
    fit_plot.data_source.data['x'] = x
    fit_plot.data_source.data['y'] = rv.pdf(x)
    
    hist_fig.title.text = elec_def[elec_regions[plot_chan].astype(str)] + ': f=%.1fHz, p=%.4f '%(f,lf.ks_pvals[plot_chan,plot_freq])
    push_notebook()

In [55]:
lf = lfpca_all[0]
y, x = np.histogram(lf.spg[0,10,:], bins=20, density=True)
hist_fig = figure(plot_height=400, plot_width=400, x_axis_label='Power', y_axis_label='Probability',background_fill_color="#E8DDCB")
hist_plot = hist_fig.quad(top=y, bottom=0, left=x[:-1], right=x[1:], fill_color="#036564", line_color="#033649")
fit_plot = hist_fig.line(x[:-1],y, line_width=8,alpha=0.7,line_color="#D95B43",legend='Fit PDF')
show(hist_fig, notebook_handle=True)
interact(update_spct_hist, f=(1,100), numbins=range(10,55,5), channel=chan_sel)

C:\Users\Lauren\Anaconda3\lib\site-packages\bokeh\models\sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 21), ('y', 20)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


<function __main__.update_spct_hist>

In [42]:
y, x = np.histogram(lf.spg[0,10,:], bins=20, density=True)

In [37]:
?np.histogram